In [1]:
path_file = "../data/COLIEE2024statute_data-English/train/"

In [2]:
from os import listdir
from os.path import isfile, join

def get_all_files_from_path(mypath):
    filenames = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    return filenames

from bs4 import BeautifulSoup
import re
import json

def get_article(articles):
    result = {}
    current_statue = "(non-statute)"
    for i in re.split(r"(.*)", articles.strip()):
        if len(i) == 0 or i == "\n":
            continue
        if re.search(r"^\(.*\)$", i):
            current_statue = i.strip()
            if current_statue not in result:
                result.update({current_statue: []})
        else:
            if current_statue not in result:
                result.update({current_statue: []})
            result[current_statue].append(i)
    return result

def build_test(filename):
    result = {}
    with open(filename, 'r') as f:
        data = f.read()

    data = BeautifulSoup(data, "xml").find_all('pair')
    for i in data:
        id = i.get('id')
        result.update({id: {}})
        result[id].update({"label": i.get('label')})
        articles = i.find('t1').text.strip()
        # articles = get_article(articles)
        result[id].update({"result": articles})
        result[id].update({"content": i.find('t2').text.strip()})
    return result

def write_json(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

import xml.etree.ElementTree as Et
import glob

def format_first_line(text):
    lines = text.split("\n")
    results = []
    for line in lines:
        if line[0] == "":
            continue
        if line[0] == "(" and line[-1] == ")":
            continue
        results.append(line)
    return "\n".join(results)

def load_samples(filexml):
    # try:
        tree = Et.parse(filexml)
        root = tree.getroot()
        samples = []
        for i in range(0, len(root)):
            sample = {'result': []}
            for j, e in enumerate(root[i]):
                if e.tag == "t1":
                    sample['result'] = format_first_line(e.text.strip())
                elif e.tag == "t2":
                    question = e.text.strip()
                    sample['content'] = question if len(question) > 0 else None
            sample.update(
                {'index': root[i].attrib['id'], 'label': root[i].attrib.get('label', "N")})
            # filter the noise samples
            if sample['content'] is not None:
                samples.append(sample)
            else:
                print("[Important warning] samples {} is ignored".format(sample))
        return samples

def load_test_data_samples(path_folder_base, test_id):
    data = []
    test = load_samples(f"{path_folder_base}/riteval_{test_id}.xml")
    for file_path in glob.glob(f"{path_folder_base}/riteval_{test_id}.xml"):
        data = data + load_samples(file_path)
    return data


def load_all_data_samples(path_folder_base):
    data = []
    for file_path in glob.glob("{}/*.xml".format(path_folder_base)):
        data = data + load_samples(file_path)
    return data

def check_false_labels(pred, false_labels):
	for label in false_labels:
		if label in pred:
			return True
	return False

from tqdm import tqdm

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip().lower()

def readfile(filename):
    f = open(filename)
    data = json.load(f)
    return data

# ZeroShot_CoT

In [3]:
from transformers import AutoTokenizer, BloomForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-xxl"
cache_dir = "/home/congnguyen/drive/.cache"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(
		model_name, device_map="auto", cache_dir=cache_dir, torch_dtype=torch.float16, load_in_8bit=True
	)

/home/congnguyen/drive/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.29s/it]


In [4]:
from tqdm import tqdm

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip().lower()

def prompting(premise, hypothesis, template=None):
    text = template.replace("{{premise}}", premise).replace("{{hypothesis}}", hypothesis)
    return text+"\n\nLet's think step by step\n"

In [5]:
list_prompt = readfile("../data/prompt.json")
# list_prompt

In [6]:
# def predict(model, tokenizer, files=["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], output="../output/cot/newpromt_"):
#     for file in files:
#         test_file = path_file+file+".xml"
#         data = load_samples(test_file)
        
#         acc = {}
#         for template_prompt in list_prompt:
#             template_prompt = template_prompt["prompt"]
#             print(template_prompt)
#             result = []
#             count = 0
#             for item in tqdm(data):
#                 label = item["label"]
#                 hypothesis = item["content"]
#                 premise = item["result"]
#                 #Important: You must use dot-product, not cosine_similarity
#                 text = prompting(premise, hypothesis, template_prompt)
#                 inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
#                 outputs = model.generate(inputs, max_new_tokens=256)
#                 output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
#                 if count < 10:
#                     print(text)
#                     print(output_text)
#                 # if count < 3:
#                 #     print(output_text)
#                 if "yes" in output_text or "true" in output_text:
#                     output_text = "Y"
#                 else:
#                     output_text = "N"
#                 if output_text == label:
#                     count+=1
#                 # print("predict label: ", output_text, "label: ", label)
#             print("=======================================")
#             print(template_prompt)
#             print(count, "/", len(data))
#             acc.update({template_prompt: count/len(data)})
#         writefile(acc, output+file+".json")

In [7]:
# predict(model, tokenizer, ["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], "../output/cot/newpromt_")

# Dense Passage Retrieval

In [8]:
# datas = get_all_files_from_path("../data/COLIEE2024statute_data-English/train")
# corpus = []
# content = []
# labels = []
# for data in datas:
#     data = load_samples(data)
#     for item in data:
        # corpus.append(item["result"])
        # content.append(item["content"])
        # labels.append(item["label"])
# print(len(corpus))


In [8]:
import json

def readjsonl(filename="../output/generated_cot/prompt_0/riteval_R04_en_cot.jsonl"):
    jsonl_content = open(filename, "r", encoding="utf-8")
    result = [json.loads(jline) for jline in jsonl_content.read().splitlines()]
    return result
    
def load_cot_sample(path="../output/generated_cot/prompt_0"):
    files = get_all_files_from_path(path)
    corpus = []
    prompts = []
    labels = []
    contents = []
    cots = []
    for file in files:
        data = readjsonl(file)
        for item in data:
            corpus.append(item["result"])
            contents.append(item["content"])
            prompts.append(item["prompt"])
            cots.append(item["cot"].split(".")[0]+".")
            labels.append(item["label"])
    return corpus, prompts, labels, contents, cots

In [9]:
corpus, prompts, labels, contents, cots = load_cot_sample("../output/generated_cot/prompt_0")

In [10]:
from sentence_transformers import SentenceTransformer, util
import torch

passage_encoder = SentenceTransformer('facebook-dpr-ctx_encoder-single-nq-base')

passage_embeddings = passage_encoder.encode(corpus)

query_encoder = SentenceTransformer('facebook-dpr-question_encoder-single-nq-base')

In [20]:

def fewshot_cot_prompting(indexes, prompts, labels, cots):
    result = ""
    for i in indexes:
        if "true or false" in prompts[i].lower():
            answer = "True"
            if "N" == labels[i]:
                answer = "False"
        else:
            answer = "Yes"
            if "N" == labels[i]:
                answer = "No"
        # prompt = prompt_template.replace("{{premise}}", corpus[i]).replace('{{hypothesis}}', content[i]).replace('{{answer}}', answer)
        prompt = prompting(prompts[i], cots[i], answer)
        result += prompt
    return result
    
def prompting(text, cot, answer):
    return text.split("\nAnswer: ")[0] + " Let's think step by step \nAnswer: "+ cot +" So the answer is " + answer+"\n\n"
    # return "### Instructs: "+text.split("\nAnswer: ")[0] + "\n### Response: "+ cot

In [21]:
test_file = "../data/COLIEE2024statute_data-English/train/riteval_R04_en.xml"
data = load_samples(test_file)

In [22]:
template_prompt = "{{premise}}\nQuestion: {{hypothesis}} True or False?"

count = 0
for item in tqdm(data):
    premise = item["result"]
    label = item["label"]
    hypothesis = item["content"]
    #Important: You must use dot-product, not cosine_similarity
    query_embedding = query_encoder.encode(premise)
    scores = util.dot_score(query_embedding, passage_embeddings)
    indexes = torch.topk(scores, 3).indices[0]
    prompt = fewshot_cot_prompting(indexes, prompts, labels, cots)
    text = template_prompt.replace("{{premise}}", premise).replace("{{hypothesis}}", hypothesis) +"\nAnswer: True or False"
    text = prompt+text
    if count < 5:
        print(text)
        print("===================================")
    inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
    outputs = model.generate(inputs, max_new_tokens=256)
    output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
    print(output_text)
    print("===================================")
    if "yes" in output_text or "true" in output_text:
        output_text = "Y"
    else:
        output_text = "N"
    print("Predict: ", output_text, " Label: ", label)
    if output_text == label:
        count+=1

  0%|                                                                                                                 | 0/101 [00:00<?, ?it/s]

Article 5 (1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.
(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.
Article 6 (1) A minor who is permitted to conduct one or multiple types of business has the same capacity to act as an adult as far as that business is concerned.
Question: A juridical act by a minor who is permitted to conduct business may not be rescinded even if the judicial act does not concern the business. True or False? Let's think step by step 
Answer: A juridical act in contravention of the provisions of the preceding paragraph is voidable. So the answer is False

Article 6
(1) A minor who is permitted to conduct one or multiple types of business has the same capacity to act as an adult as far as that business is concerned.
(2) In a case as referred

  1%|█                                                                                                        | 1/101 [00:04<07:55,  4.76s/it]

let's think step by step answer: a juridical act in contravention of the provisions of the preceding paragraph is voidable. so the answer is false
Predict:  N  Label:  N
Article 5 (1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.
(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.
Article 120 (1) An act that is voidable on the grounds of the qualified legal capacity to act of the person who did the act may be rescinded only by the person with qualified legal capacity (in the case of an act performed by the person as a legal representative of another person with limited capacity, including that other person with limited capacity), or an agent or successor thereof, or a person who has the authority to give consent thereto.
Question: When a minor concludes a contract wit

  2%|██                                                                                                       | 2/101 [00:12<10:38,  6.45s/it]

...

let's think step by step answer: the possessor must be in bad faith for the court to grant a reasonable period of time for the reimbursement. so the answer is false
Predict:  N  Label:  N


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 100/101 [14:29<00:08,  8.18s/it]

let's think step by step answer: the pledgee may make a request to the court seeking the immediate appropriation of the thing pledged for the satisfaction of that claim in accordance with the evaluation of an appraiser only when there are reasonable grounds. so the answer is false
Predict:  N  Label:  N


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [14:40<00:00,  8.71s/it]

let's think step by step answer: the performer may, with the permission of the court, sell the thing that is the subject matter of performance at public auction and deposit the proceeds of such sales with an official depository: (iii) excessive expenses are required for the preservation of the thing; or. so the answer is true
Predict:  Y  Label:  Y


In [27]:
def predict(model, tokenizer, path_file, passage_embeddings, files=["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], output="../output/accuracy2/newpromt_"):
    for file in files:
        test_file = path_file+file+".xml"
        f = open(path_file+file+".txt", "w", encoding="utf-8")
        data = load_samples(test_file)
        
        acc = {}
        for template_prompt in list_prompt:
            idx = template_prompt["id"]
            template_prompt = template_prompt["prompt"]
            corpus, prompts, labels, contents, cots = load_cot_sample("/home/congnguyen/drive/Coliee2024/output/generated_cot/prompt_"+str(idx))
            result = []
            count = 0
            for item in tqdm(data):
                premise = item["result"]
                label = item["label"]
                hypothesis = item["content"]
                #Important: You must use dot-product, not cosine_similarity
                query_embedding = query_encoder.encode(premise)
                scores = util.dot_score(query_embedding, passage_embeddings)
                indexes = torch.topk(scores, 3).indices[0]
                prompt = fewshot_cot_prompting(indexes, prompts, labels, cots)
                if "true or false" in template_prompt.lower():
                    text = template_prompt.replace("{{premise}}", premise).replace("{{hypothesis}}", hypothesis) +"\nAnswer: True or False"
                else:
                    text = template_prompt.replace("{{premise}}", premise).replace("{{hypothesis}}", hypothesis) +"\nAnswer: Yes or No"
                text = prompt+text
                if count < 5:
                    print(text)
                    print("===================================")
                inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
                outputs = model.generate(inputs, max_new_tokens=256)
                output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
                print(output_text)
                print("===================================")
                if "yes" in output_text or "true" in output_text:
                    output_text = "Y"
                else:
                    output_text = "N"
                print("Predict: ", output_text, " Label: ", label)
                if output_text == label:
                    count+=1
                # print("predict label: ", output_text, "label: ", label)
            # print("=======================================")
            # print(template_prompt)
            # print(count, "/", len(data))
            acc.update({template_prompt: count/len(data)})
            print(acc)
        writefile(acc, output+file+".json")
        f.close()


In [ ]:
predict(model, tokenizer, path_file, passage_embeddings, ["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], "../output/newpromt_")


  0%|                                                                                                                 | 0/111 [00:00<?, ?it/s]

Article 5
(1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.
(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.
(3) Notwithstanding the provisions of paragraph (1), a minor may freely dispose of property that the legal representative has permitted the minor to dispose of for a specified purpose, to an extent that falls within the scope of that purpose. The same applies if the minor disposes of property that the legal representative has permitted the minor to dispose of without specifying a purpose.
Question: A contract of sales concluded by a minor may not be rescinded if it relates to daily life, even in cases the consent of the parental authority is not obtained. True or False? Let's think step by step 
Answer: Article 5 (1) A minor must obtain the consent of the mi

  1%|▉                                                                                                        | 1/111 [00:20<36:57, 20.16s/it]

let's think step by step answer: article 5 (1) a minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation. (2) a juridical act in contravention of the provisions of the preceding paragraph is voidable. (3) notwithstanding the provisions of paragraph (1), a minor may freely dispose of property that the legal representative has permitted the minor to dispose of for a specified purpose, to an extent that falls within the scope of that purpose. the same applies if the minor disposes of property that the legal representative has permitted the minor to dispose of without specifying a purpose.
Predict:  N  Label:  N
Article 9
A juridical act performed by an adult ward is voidable;provided, however, that this does not apply to the purchase of daily necessities or to any other act involved in day-to-day life.
Question: An  act of pa

  2%|█▉                                                                                                       | 2/111 [00:27<23:23, 12.87s/it]

...

let's think step by step answer: the obligee(b) may perform the obligation which he/she contracts with a third party by assigning the claim which the obligor(a) contracts with the obligee(b) to the third party as a substitute against the will of the obligor(a). so the answer is false
Predict:  N  Label:  N


 63%|█████████████████████████████████████████████████████████████████▌                                      | 70/111 [09:44<06:01,  8.82s/it]

let's think step by step answer: the assignment of the credit of sale value is not a requirement for assertion of the assignment. so the answer is false
Predict:  N  Label:  N


 64%|██████████████████████████████████████████████████████████████████▌                                     | 71/111 [09:57<06:49, 10.23s/it]

let's think step by step answer: the tender of the performance must be made actually consistent with the main purport of the obligation;provided, however, that if the obligee refuses to accept that performance in advance or if any act is required on the part of the obligee with respect to the performance of the obligation, it is sufficient for the obligor to request the acceptance thereof by giving a notice that the tender of the performance has been prepared. so the answer is true
Predict:  Y  Label:  Y


In [75]:
cots

["A must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.",
 'A may freely dispose of property that the legal representative has permitted the minor to dispose of for a specified purpose, to an extent that falls within the scope of that purpose.',
 "The family court may decide to commence a guardianship in respect of a person who constantly lacks the capacity to appreciate the person's own situation due to a mental disorder, at the request of the person in question, that person's spouse, that person's relative within the fourth degree of kinship, the person's guardian of a minor, the person's supervisor of a minor's guardian, the person's curator, the person's curator's supervisor, the person's assistant, the person's assistant's supervisor, or a public prosecutor.",
 "A must obtain the consent of the minor's legal representa

In [16]:
94/101

0.9306930693069307